<a href="https://colab.research.google.com/github/mollyapsel/RegionalLinguisticBias/blob/main/filter_texts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/Michelle Molly/Corpus Data/USA Text/Clean_text.zip'

Archive:  /content/drive/MyDrive/Michelle Molly/Corpus Data/USA Text/Clean_text.zip
   creating: Clean_text/
  inflating: Clean_text/14-01-us.json  
  inflating: Clean_text/14-11-us.json  
  inflating: Clean_text/20-03-us4.json  
  inflating: Clean_text/21-07-us3.json  
  inflating: Clean_text/20_06-us5.json  
  inflating: Clean_text/11-10-us.json  
  inflating: Clean_text/20_07-us5.json  
  inflating: Clean_text/23-08-us3.json  
  inflating: Clean_text/22-04-us1.json  
  inflating: Clean_text/22-05-us1.json  
  inflating: Clean_text/15-07-us.json  
  inflating: Clean_text/21-01-us4.json  
  inflating: Clean_text/23-06-us1.json  
  inflating: Clean_text/23-07-us1.json  
  inflating: Clean_text/20-04-us3.json  
  inflating: Clean_text/20-05-us3.json  
  inflating: Clean_text/22-10-us5.json  
  inflating: Clean_text/19-07-us4.json  
  inflating: Clean_text/10-06-us.json  
  inflating: Clean_text/22-11-us5.json  
  inflating: Clean_text/21-09-us1.json  
  inflating: Clean_text/21-08-us1.j

In [ ]:
import json
import pandas as pd

#sources = pd.read_csv('/content/drive/MyDrive/Michelle Molly/Corpus Data/Tagged Sources/merged_output.csv')
#sources.head()


In [ ]:
!pip install uszipcode

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.9 MB/s eta 0:00:00
  Created wheel for atomicwrites: filename=atomicwrites-1.4.1-py2.py3-none-any.whl size=6940 sha256=86045d225574ecca12c1c25d35002bce4a60e18750f96c4ce22fb87cc357ab6a
  Stored in directory: /root/.cache/pip/wheels/34/07/0b/33b15f68736109f72ea0bb2499521d87312b932620737447a2
Successfully built atomicwrites


In [ ]:
!pip install sqlalchemy-mate==2.0.0.0

In [ ]:
from uszipcode import SearchEngine

search = SearchEngine()

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Download /root/.uszipcode/simple_db.sqlite from https://github.com/MacHu-GWU/uszipcode-project/releases/download/1.0.1.db/simple_db.sqlite ...
  1.00 MB downloaded ...
  2.00 MB downloaded ...
  3.00 MB downloaded ...
  4.00 MB downloaded ...
  5.00 MB downloaded ...
  6.00 MB downloaded ...
  7.00 MB downloaded ...
  8.00 MB downloaded ...
  9.00 MB downloaded ...
  10.00 MB downloaded ...
  11.00 MB downloaded ...
  Complete!


In [ ]:
from tqdm import tqdm
total = 0
badzip = 0
validzip = 0
typo = 0
for zip in tqdm(sources['zip_code'].values):
  if pd.isna(zip):
    continue
  zip = int(zip)
  total += 1
  if search.by_zipcode(zip) is None:
    badzip += 1
    for i in range(10):
      if search.by_zipcode(str(zip) + str(i)) is not None:
        typo += 1
        break
  else:
    validzip += 1

print("Valid zip:", validzip)
print("Bad zip:", badzip)
print("Found alternative:", typo)
print("Total:", total)


100%|██████████| 4788730/4788730 [25:28<00:00, 3133.14it/s]

Valid zip: 1977060
Bad zip: 78490
Found alternative: 14826
Total: 2055550


In [ ]:
type(sources['ID'].values[0])

numpy.int64

In [ ]:
sources.loc[sources['zip_code'] == 7520, 'zip_code'] = 75201
sources.loc[sources['zip_code'] == 1809, 'zip_code'] = 18042



In [ ]:
import os
from tqdm import tqdm
state_collections = {}
directory = '/content/Clean_text'

for file in tqdm(os.listdir(directory)):
  with open(os.path.join(directory,file)) as f:
      d = json.load(f)
      #print(d)

  for x in d:
    id = int(x['text_id'])
    if id in sources['ID'].values:
      # get the value of the zip_code column in the row with the matching ID
      zip_code = sources.loc[sources['ID'] == id, 'zip_code'].iloc[0]
      # if zip_code is not NaN
      if pd.isna(zip_code):
        continue
      zip_code = int(zip_code)
      if search.by_zipcode(zip_code) is None:
        for i in range(10):
          if search.by_zipcode(str(zip_code) + str(i)) is not None:
            zip_code = str(zip_code) + str(i)
            break
        else:
          continue

      x['zip_code'] = zip_code
      state = search.by_zipcode(zip_code).state
      if state in state_collections:
        state_collections[state].append(x)
      else:
        state_collections[state] = [x]

for state in state_collections:
  with open('/content/drive/MyDrive/Michelle Molly/Corpus Data/States/'+ state + '.json', 'w') as f:
    json.dump(state_collections[state], f)




 25%|██▌       | 101/398 [6:02:27<11:00:05, 133.35s/it]

In [ ]:
state_collections.keys()

dict_keys(['NY', 'CA', 'GA', 'ME', 'TX', 'IL', 'DC', 'TN', 'OR', 'MN', 'MI', 'CO', 'IN', 'PA', 'MT', 'NE', 'FL', 'UT', 'NJ', 'CT', 'LA', 'WA', 'WI', 'MA', 'AL', 'OH', 'VA', 'MO', 'MD', 'WY', 'RI', 'IA', 'DE', 'AE', 'HI', 'NC', 'SC', 'KY', 'WV', 'NM', 'OK', 'AZ', 'AR', 'ID', 'NH', 'NV', 'KS', 'ND', 'SD', 'VT', 'MS', 'AK'])

In [ ]:
state_ct = {'MO': 14345754, 'MN': 34208180, 'MI': 24463150, 'ME': 5113440, 'MD': 19819691, 'VT': 1427418, 'MA': 32309863, 'GA': 20071155, 'CT': 16918178, 'CO': 15667506, 'VA': 17856848, 'PA': 16953564, 'FL': 31124180, 'UT': 3773832, 'LA': 5748383, 'KY': 4463012, 'KS': 2769992, 'TX': 39664609, 'AZ': 13307110, 'TN': 13320177, 'AR': 4686714, 'AL': 2630784, 'AK': 3191779, 'AE': 393185, 'DE': 1055497, 'DC': 91253387, 'SD': 2743912, 'SC': 4336728, 'OR': 7043155, 'OK': 4465267, 'OH': 13026265, 'IN': 19898825, 'IL': 35740392, 'ID': 1535155, 'IA': 4692954, 'CA': 157794735, 'RI': 1293576, 'NY': 237432441, 'NV': 4743654, 'NM': 5581678, 'NJ': 12729183, 'NH': 2176231, 'WY': 665879, 'NE': 4643979, 'ND': 2154060, 'WV': 2555250, 'NC': 7451717, 'HI': 2655708, 'WI': 6604489, 'WA': 15700395, 'MT': 3271443, 'MS': 1391553}

In [ ]:
state_ct['MT']+state_ct['WY']+state_ct['CO']+state_ct['ID']

21139983

In [ ]:
state_ct['CT']

16918178

In [ ]:
for st in state_ct:
  if state_ct[st] > 15000000:
    print(st)

MN
MI
MD
MA
GA
CT
CO
VA
PA
FL
TX
DC
IN
IL
CA
NY
WA


In [ ]:
for st in state_ct:
  if state_ct[st] < 1000000:
    print(st, state_ct[st])

AE 393185
WY 665879


In [ ]:
len(state_ct.keys())

52

# Collect zips for congressional mapping

In [ ]:
import os
directory =